# 1. Property Prediction - 化合物の水溶解度予測

分子構造は、元素とその位置、つまり3次元立体構造データとして表現されます。本ノートブックでは、DGL-LifeSciを用いて、分子構造のデータをそのまま入力して物質の物性値を予測するタスク（Property Prediction）を行います。
- 本ノートブックでは、分子構造に基づいて低分子化合物の水への溶解度予測を行います。本章で扱うデータは、ESOLのデータセットです。[ESOL（Estimated SOLubility）](https://pubs.acs.org/doi/10.1021/ci034243x)のデータセットは、化合物の水への溶解度に関するパブリックデータセットです（元論文の著者名から「Delaneyデータセット」とも呼ばれることがあります）。[MoleculeNet](https://pubs.rsc.org/en/content/articlelanding/2018/sc/c7sc02664a)と呼ばれる分子機械学習ベンチマークデータセットのうちの一つで、1128の化合物のデータが含まれています。
- 今回扱うタスクは、入力となる化合物の水への溶解度の数値を予測する問題（回帰問題, Regression Task）となります。

## 1.1. Using pre-trained model
- 例えば「試しに予測してみたい化合物はあるけれどGNNを1から学習させるのはハードルが高い...」という方もいらっしゃるかと思います。
- DGL-LifeSciではいくつかの有名なデータセットを用いて事前に学習されたモデルを提供しています。（一覧はこちら https://lifesci.dgl.ai/api/model.pretrain.html#api ）
- 1.1では事前学習済みモデルを用いることで、手元で機械学習のトレーニングを行うことなく、すぐに分子特性予測を行うことを体感します。

### Requirements
DGL-LifeSciおよび必要なライブラリをインストールします。
- DGL-LifeSci
- Python 3.6+
- PyTorch 1.5.0+
- DGL 0.7.0+
- RDKit 2018.09.3

In [ ]:
!pip install dgl
!pip install dgllife
!conda install -c rdkit -y rdkit==2018.09.3

### Inference with pre-trained model
ESOLのデータセットで事前に学習されたモデルをロードします。

In [ ]:
import torch
from dgllife.data import ESOL
from dgllife.model import load_pretrained
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer

model = load_pretrained('GCN_canonical_ESOL') # Pretrained model loaded

ではロードしたモデルの構造を見てみましょう。 （ちなみに本来`model.eval()`は、モデルを推論モードに切り替えるためのものです。）

In [ ]:
model.eval()

次にデータセットをロードします。ここには学習に用いられた1128個の化合物に関するデータが含まれています。

In [ ]:
dataset = ESOL(smiles_to_bigraph, node_featurizer=CanonicalAtomFeaturizer())

試しに、ロードしたデータセットの中から一つ選び、中身を見てみましょう。

In [ ]:
smiles, g, label = dataset[0]
print('SMILES: ', smiles)
print('DGLGraph: ', g)
print('Node Features: ',g.ndata['h'], g.ndata['h'].size())
print('Label: ', label)

各出力は以下のような意味となります。
- SMILES：対象としている化合物をSMILES記法で表した文字列
- DGLGraph：化合物をグラフ形式（DGLGraph）で表したオブジェクト
    - Node Features： 分子内の原子（グラフにおけるノード）に関係する特徴量の集まり。その原子が何か、隣接する水素原子の数など、各原子に関わる情報を数値化したものになります。（今回使用している`CanonicalAtomFeaturizer`についてはこちらをご覧ください https://lifesci.dgl.ai/generated/dgllife.utils.CanonicalAtomFeaturizer.html)
- Label: 予測すべき値となる水溶解度(logS = $\log_{10} \frac{mol}{L}$)を表しています。

選択された化合物の構造をRDKitを用いて確認してみましょう。

In [ ]:
from IPython.display import SVG
from rdkit import Chem
from rdkit.Chem import Draw

mol = [Chem.MolFromSmiles(smiles)]
SVG(Draw.MolsToGridImage(mol, molsPerRow=1, useSVG=True))

では、選択された化合物に対して、事前に学習されたモデルを用いて水溶解度を予測していきます。

In [ ]:
# Pop the node features
feats = g.ndata.pop('h')

# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    label_pred = model(g, feats)

# Mask non-existing labels
print(label_pred)

## 1.2. Train your own model on a CSV Dataset

- 実際に分子特性予測を行う場合、皆さんがそれぞれお持ちのカスタムデータセットを使ったモデルの作成を検討されると思います。
- DGL-LifeSciには、学習に使いたいデータ（CSVファイル）で用意していただければ、コマンドライン一行でモデルの学習 / 推論ができるプログラムが用意されています。
- 本節ではDGL-LifeSciが提供するプログラムを用いて化合物の水溶解度を予測するモデルの学習・推論を行います。

本ハンズオンでは、化合物の水溶解度に関するCSVデータとして、1.1でも使用したESOLデータセットを使用します。もしお手持ちのデータでモデルの学習ないし推論を試したい場合は、以下で説明する形式のCSVファイルを作成してください。

### Prepare a CSV Dataset

まずは対象データをCSV形式で準備する必要があります。ESOLデータセットをダウンロードします。

In [ ]:
from dgl.data.utils import download, _get_dgl_url, extract_archive

url = 'dataset/ESOL.zip'
data_path = 'ESOL.zip'
download(_get_dgl_url(url), path=data_path)
extract_archive(data_path, './ESOL')

In [ ]:
import pandas as pd

df = pd.read_csv('ESOL/delaney-processed.csv')
df.head(10)

ダウンロードしたデータは上のようになっています。
複数あるカラムのうち、今回予測対象となる水溶解度は`measured log solubility in mols per litre`であり、`smiles`カラムが入力となるSMILES文字列となります。

### Download sample scripts

コマンドラインで実行できるプログラムはDGL-LifeSciの公式リポジトリにあります。まず、リポジトリをクローンします。

In [ ]:
! git clone https://github.com/awslabs/dgl-lifesci.git

In [ ]:
%cp ./dgl-lifesci/examples/property_prediction/csv_data_configuration/ -r ./

### Quick Analysis

まずはデータセットについて分析してみましょう。化合物について簡単に分析するためのスクリプトが用意されています。

In [ ]:
!python ./csv_data_configuration/analysis.py -c ./ESOL/delaney-processed.csv -sc 'smiles' -p ./analysis_results

In [ ]:
!cat ./analysis_results/summary.txt

### Train and Evaluate

In [ ]:
%cd ./csv_data_configuration/
!python ./regression_train.py -c ../ESOL/delaney-processed.csv -sc 'smiles' -t 'measured log solubility in mols per litre' -me rmse -nw 0 -p ../regression_results

よりモデル性能を向上させるため、モデルのハイパーパラメータの最適化を行います。
`-ne <num_trials>`のオプションを加えることで、`<num_trials>`で指定した回数分ハイパーパラメータの探索(ベイズ最適化)を行うことができます。

In [ ]:
# With Hyper Parameter Search
%cd ../csv_data_configuration/
!python regression_train.py -ne 5 -c ../../../../ESOL/delaney-processed.csv -sc 'smiles' -t 'measured log solubility in mols per litre' -me rmse -nw 0 -p ../../../../regression_results

#### ToDo 
テストセット用CSVファイルを別に作って残しといてそれで推論

## 1.3 Train your own models using Amazon SageMaker Training Job

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session


sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'dgllifesci/esol/dataset'

role = get_execution_role()

In [ ]:
# データをS3にアップロード
%cd ../
!aws s3 cp ./ESOL/delaney-processed.csv s3://{bucket}/{prefix}/delaney-processed.csv
!aws s3 ls s3://{bucket}/{prefix}/

In [ ]:
s3_input = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/delaney-processed.csv', content_type="csv")

In [ ]:
!touch dgl-lifesci/examples/property_prediction/csv_data_configuration/requirements.txt
!echo "dgl-cu101 -f https://data.dgl.ai/wheels/repo.html\ndgllife\nrdkit-pypi" > dgl-lifesci/examples/property_prediction/csv_data_configuration/requirements.txt
!cat dgl-lifesci/examples/property_prediction/csv_data_configuration/requirements.txt

In [ ]:
from sagemaker.pytorch import PyTorch
# import json
# # JSON encode hyperparameters
# def json_encode_hyperparameters(hyperparameters):
#     return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

# hyperparameters = json_encode_hyperparameters(
#     {
#         'dataset':'Tox21',
#         'model':'GCN',
#         'featurizer-type':'canonical',
#         'sagemaker_program':'classification.py',
#         'sagemaker_submit_directory':inputs
#     }
# )

hyperparameters = {
    'csv-name': 'delaney-processed.csv',
    'model':'GCN',
    'atom-featurizer-type':'canonical',
    'smiles-column':'smiles',
    'task-names':"measured log solubility in mols per litre",
    'metric':'rmse',
    'num-workers': 0,
    'num-evals': 10,   
}

# Create estimator
estimator = PyTorch(
    entry_point="regression_train_sagemaker.py",
    source_dir="./csv_data_configuration/",
    role=role,
    framework_version="1.6.0",
    py_version="py3",
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters=hyperparameters
)

In [ ]:
# 学習を開始
estimator.fit({'train':s3_input})

In [ ]:
estimator.latest_training_job.job_name
# !aws s3 cp {estimator.output_path}{estimator.latest_training_job.job_name}/output/model.tar.gz .

## 1.4. Train your own models using Amazon SageMaker Training Job with Custom Container (※ Optional) 

#### メモ
- requirementの`RDKit 2018.09.3`がconda installでないとinstallできないので、document通りに確実に動く環境を整えるならこっち（のはず）。
- ただし`pip install rdkit-pypl`でも対応できる(2022年12月現在)。version 2018.09.3のものはサポートされていないが、dglaiの方のハンズオンではこれで対応している。

### Setup
SageMakerで学習を実行するにあたり必要なライブラリをインストールします。

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session


sess = sagemaker.Session()
bucket = sess.default_bucket()

role = get_execution_role()

In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
%%sh

docker_name=sagemaker-dgllifesci-py36
sm-docker build . -f ./container/Dockerfile --repository $docker_name:1.0

In [ ]:
!tar -czvf source.tar.gz utils.py classification.py ./configures

In [ ]:
prefix = 'dgl-lifesci/property_prediction/code'
inputs = sess.upload_data(path='./source.tar.gz', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

In [ ]:
import json
# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters(
    {
        'dataset':'Tox21',
        'model':'GCN',
        'featurizer-type':'canonical',
        'sagemaker_program':'classification.py',
        'sagemaker_submit_directory':inputs
    }
)

estimator = sagemaker.estimator.Estimator(
    image_uri = '233488627969.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-dgllifesci-py36:1.0',
    role = role,
    instance_count = 1,
    instance_type = "ml.m5.4xlarge",
    hyperparameters = hyperparameters,
    sagemaker_session = sess
)
estimator.fit()